### Visualize the MRI images and the segmentation results of the tumors.
- MRI images: t1, t1Gd, t2，flair.
- Tumor segmentations: 
                {"necrotic (NCR)": 1, 
                "peritumoral edematous/invaded tissue (ED)": 2,
                "enhancing tumor(ET)": 4,
                "tumor core (TC)": [1,4],
                "wholeTumor": [1, 2, 4]}

In [ ]:
import os
import numpy as np
import sys
import gc
import matplotlib.pyplot as plt

import nibabel as nib

sys.path.append("../")
from utils.myUtils import traversalDir_FirstDir, mkdir
from mySettings import get_image_visualization_setting_dict



In [ ]:
def get_maximum_segmentation_slice_index(image_array):
    shape=image_array.shape
    count_non_zero_list=[]
    for index in range(shape[2]):
        image_slice=image_array[:, :, index]
        count_non_zero=np.count_nonzero(image_slice)
        count_non_zero_list.append(count_non_zero)
   
    max_nonzero_count=max(count_non_zero_list)
    index=count_non_zero_list.index(max_nonzero_count)
    print("max_nonzero_count={}, index={}".format(max_nonzero_count, index))
    
    return index

def show_one_patient(image_base_folder, segmentation_folder, patient_id, modality_list, save_visualization_folder):

    sub_figure_number=len(modality_list) #len(modality_list)+1
    plt.figure(figsize=(5*sub_figure_number,5))
    
    # get the slice index with maximum non-zero pixels.
    segmentation_path=os.path.join(segmentation_folder, patient_id+".nii.gz")
    segmentation_array=nib.load(segmentation_path).get_fdata()
    index=get_maximum_segmentation_slice_index(segmentation_array)
    
    # plot MRI images with different modalities       
    for i, modality in enumerate(modality_list):
        image_path=os.path.join(image_base_folder, patient_id, patient_id+"_"+ modality+".nii.gz")
        image_array=nib.load(image_path).get_fdata()
        image_slice=image_array[:, :, index].T
        sub_fig=plt.subplot(1, sub_figure_number, i+1)
        sub_fig.set_title(patient_id+" ("+modality+")", fontsize=18)
        plt.axis('off')
        plt.imshow(image_slice, cmap='gray')


#     # plot segmentation
#     segmentation_slice=segmentation_array[:, :, index].T
#     sub_fig=plt.subplot(1, sub_figure_number, sub_figure_number)
#     sub_fig.set_title("segmentation", fontsize=18)
#     plt.axis('off')
#     plt.imshow(image_slice, aspect='auto',cmap='gray')
#     plt.imshow(segmentation_slice, alpha=0.8)
    
    #save visualization results.
    plt.subplots_adjust(left=0.03, bottom=0.03, right=0.97, top=0.9,  wspace=0.03, hspace=0)
    save_path= os.path.join(save_visualization_folder, patient_id+"_slice_"+str(index)+".jpeg")
    plt.savefig(save_path)
    plt.show()
    
    # Clear the current axes.
    plt.cla() 
    # Clear the current figure.
    plt.clf() 
    # Closes all the figure windows.
    plt.close('all')
    gc.collect()        
    
def visualize_image_and_segmentations(image_base_folder, segmentation_folder, modality_list, save_visualization_folder):
    
    patient_id_list=traversalDir_FirstDir(image_base_folder)
    for patient_id in patient_id_list:
        show_one_patient(image_base_folder, segmentation_folder, patient_id, modality_list, save_visualization_folder)
        
    print("Finish saving visualization results in {}...".format(save_visualization_folder))

In [ ]:
image_visualization_setting_dict=get_image_visualization_setting_dict()
for setting_name, image_visualization_setting in image_visualization_setting_dict.items():
    image_base_folder=image_visualization_setting["image_folder"]
    segmentation_folder=image_visualization_setting["segmentation_folder"]
    modality_list=image_visualization_setting["modality_list"]
    save_visualization_folder=image_visualization_setting["save_visualization_folder"]
    mkdir(save_visualization_folder)
    visualize_image_and_segmentations(image_base_folder, segmentation_folder, modality_list, save_visualization_folder)